In [2]:
# Install and import appropriate packages and print "Done." when this is complete.

!conda install -c conda-forge scikit-learn --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

import numpy as np
import pandas as pd
import json
import lxml

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


import folium

print("Done.")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Done.


In [3]:
# Scrape wikipedia to create a dataframe with rows missing 'Borough' values removed.

wiki_dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#filtered_columns = ["PostalCode","Borough","Neighborhood"]
#wiki_df = wiki_df.loc[:, filtered_columns]
wiki_df = pd.concat(wiki_dfs)
wiki_df = wiki_df[["Postal Code","Borough","Neighbourhood"]]
wiki_df.replace(to_replace="Not assigned", value=np.NaN, inplace=True)
wiki_df.dropna(axis=0, subset=["Borough"], inplace=True)
wiki_df.reset_index(drop=True)

# Set 'Neighborhood' values to equal 'Borough' values when missing.
wiki_df['Neighbourhood'].fillna(wiki_df['Borough'])

# Shape of dataframe

wiki_df.shape

(103, 3)

In [4]:
lat_lng_df = pd.read_csv("https://cocl.us/Geospatial_data")
lat_lng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# Merge Lat/long DF with wiki DF

merged_df = wiki_df.merge(lat_lng_df,how='left')
merged_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [8]:
# Map of Downtown Toronto using latitude and longitude values
latitude = 43.665860
longitude = -79.383160

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto